# RAG_Challenge项目学习

## Main
click: 一个用于快速创建命令行工具的 Python 库。它负责解析你在终端输入的参数（如 --config）。

在数据集目录下 python main.py parse-pdfs --max-workers 4

# Pipe Line模块

### PipelineConfig:

管理所有的文件路径。它根据传入的 root_path（数据根目录）自动计算出所有中间文件和结果文件的存放位置。

### RunConfig

运行的一些设置

 - use_serialized_tables: 是否开启表格序列化（把表格转文字）。

 - parent_document_retrieval: 是否开启父文档检索（检索小块，返回大块）。

 - llm_reranking: 是否开启 LLM 重排序（用 GPT-4o 给检索结果打分）。

 - api_provider & answering_model: 选择用 OpenAI、IBM 还是 Gemini。

 - top_n_retrieval: 初步检索多少个片段。

### 解析pdf

- download_docling_models：下载 Docling 需要的模型权重

- parse_pdf_reports_parallel：并行解析pdf

# PDF_parsing PDF解析转换为json

使用docling进行解析

### class PDFParser:

- def _process_chunk ：并行处理函数
- def _parse_csv_metadata ：读取 CSV
- def _create_document_converter：配置 Docling，配置函数
- def convert_documents: 执行转换
- def process_documents 处理docling的原始结果
- def _normalize_page_sequence 标准化页码序列，确保页码连续，如果中间有缺页（比如空白页被跳过），手动补上空页对象。
- def parse_and_export : 进行解析与输出
- def parse_and_export_parallel  并行解析与输出



### class JsonReportProcessor:

这个类的作用是将 Docling 复杂的、基于引用的数据结构，转换为更直观的树状结构。

- def assemble_report：把各个部分拼起来
- def assemble_metainfo ：结合元信息并且注入公司名称
- def expand_groups：Docling 有时会把一组相关的文本（比如列表）放在一个 group 里。这个函数把 group 拆开，把里面的元素拿出来，并给它们打上 group 的标签。
- def assemble_content：将被docling分开的内容右整合在一起了，按页归档，恢复成原有的页面结构。
- def assemble_tables: 这个函数把表格转换成 LLM 能看懂的格式。遍历 Docling 解析出的所有表格对象，提取它们的元数据（位置、页码），并把表格内容同时转换为 Markdown（给 LLM 读）和 HTML（备用，描述复杂结构）。
- def assemble_pictures ：处理图片块。注意，Docling 里的“图片”不仅仅是像素图，它可能包含图片内部的文本或者图片的标题（Caption）




# class PageTextPreparation

它位于 parse-pdfs（解析）之后，chunk_reports（切片）之前。
它的任务是把 parse-pdfs 生成的那个结构化 JSON（按页码分类的各种小方块），重新**“打印”**成一份排版整齐、逻辑连贯的纯文本（Markdown 格式），以便后续切片喂给大模型。

- def process_reports :批处理pdf解析之后的json文件，目的是将每一页都进行处理，为了喂给大模型
- def process_report :处理单个报告，返回处理后的页面列表，如果有修正则打印消息。
- def prepare_page_text ：把一个个 JSON block 变成字符串
- def _clean_text ：Docling（或者它底层的 OCR）在处理某些编码奇怪的 PDF 时，会泄露字体的内部名称。比如原本写的是 2023，解析出来可能是 /two/zero/two/three.pl。这个函数就是要把这些“乱码”还原成人类可读的文字。
- def _apply_formatting_rules :为了让 RAG 效果好，文本不能随便切。语义相关的块必须在一起。比如表格的标题如果被切到上一块 chunk，而表格在下一块 chunk，模型就不知道这个表格叫什么了。这个函数通过“向前看 (i+1)”和“循环吞噬 (while)”的策略，实现了这种聚合。
- def _render_table_group ：渲染函数的作用是为了把表格或者列表相关的信息都聚合到一起，变成不可分割的一个东西。为了放在一个chunk里。防止“断章取义”
- def export_to_markdown 为了给人看，检查排版效果的，或者是给支持超长上下文的大模型



# class TextSplitter():

这段代码 src/text_splitter.py 是 RAG 系统中 Data Ingestion（数据入库） 流程的第二步，位于“合并排版”之后，“向量化”之前。

它的核心任务是：切片 (Chunking)。
它将之前整理好的、按页组织的“大块文本”，切割成适合大模型（LLM）和向量数据库处理的“小块（Chunks）”。

- def _get_serialized_tables_by_page(self, tables: List[Dict]) -> Dict[int, List[Dict]]:  把一个杂乱的表格列表，整理成一个按页码索引的字典。
- def _split_report(self, file_content: Dict[str, any], serialized_tables_report_path: Optional[Path] = None) -> Dict[str, any]: 把正文切片和表格描述切片交织在一起，给每个切片打上唯一的 ID，形成最终的切片列表。
- def _split_page ：将页面文本切分成块。原始文本包含 Markdown 表格
- def split_all_reports :获取所有待处理的json文件,然后把这些json文件都进行分块,存入文件夹






# class BM25Ingestor:

它的主要任务是：读取上一阶段（text_splitter.py）切分好的文本块（Chunks），将它们转换成机器可以检索的格式，并保存到硬盘上。实现向量数据库,使用混合检索

BM25Ingestor用于关键词检索（传统搜索)
VectorDBIngestor：用于向量检索（语义搜索，基于 FAISS 和 OpenAI Embedding）

- def create_bm25_index :从文本块列表创建 BM25 索引
- def process_reports :处理所有报告并保存独立的 BM25 索引,每个文档一个bm25索引
- def _get_embeddings: 获取向量

- def _create_vector_db 创建 FAISS 索引
- def process_reports ：处理所有报告并保存独立的 FAISS 索引,每个文档一个bm25索引

In [2]:
# qwenEmbedding：
# text-embedding-v4
import os
from openai import OpenAI

# 配置阿里云 Qwen 的信息
client = OpenAI(
        api_key="sk-734641e3a00b41d3b2b4e6f6a82c83d3",
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

try:
    print("正在尝试调用 Qwen (阿里云) 的 Embedding 接口...")
    response = client.embeddings.create(
        model="text-embedding-v4",  # 注意：这里不能填 qwen-max，必须填向量模型名
        input="测试文本"
    )
    print("✅ 成功！Qwen (阿里云) 支持 Embedding 接口。")
    print(f"向量维度: {len(response.data[0].embedding)}")
except Exception as e:
    print(f"❌ 失败: {e}")

正在尝试调用 Qwen (阿里云) 的 Embedding 接口...
✅ 成功！Qwen (阿里云) 支持 Embedding 接口。
向量维度: 1024


# class QuestionsProcessor:

他的主要作用是要进行用户问题的拆解以及分析，检索向量数据库得到相应的上下文，然后将问题+上下文喂给llm，让llm按照回答格式做出回答，返回答案

- def get_answer_for_company：通过某些公司名称，访问相应的向量数据库，然后根据检索到的上下文问大模型得到答案包含答案、引用页码的字典。
- def _load_questions ：  从 JSON 文件读取问题列表
- def _extract_references ： 提取相关引用信息，比如哪个公司，哪页
- def _validate_page_references：大模型经常胡说八道，比如明明检索了第 1 页，它非说答案在第 10 页。这个函数负责清洗模型返回的页码。去验证得到的页码是否在相应的页数里
- def _extract_companies_from_subset ：从问题文本中识别出公司名称。
- def  process_question ：问题涉及到多个公司的处理方法
- def _create_answer_detail_ref 存储思维链COT
- def process_questions_list 批处理相关问题
- def process_comparative_question 解决像 “苹果和微软谁的营收更高？” 这种问题。这类问题不能直接搜，必须拆解


# class BM25Retriever: 
基于关键词的检索
# class VectorRetriever: 
基于向量的检索 ，向量检索计算向量的cos值，查看相似度，检索出语义相似的TopN
# class HybridRetriever
这个类结合了向量检索和 LLM 重排序，旨在提高检索的准确性。


这是一个构建**检索增强生成（RAG）**系统的核心检索模块代码。它实现了三种不同的检索策略，用于从企业报告（文档）中查找与用户查询相关的上下文。

### BM25Retriever

- def  retrieve_by_company_name : 根据公司名称检索，根据关键字索引检索

### VectorRetriever

- def  _load_dbs： 加载向量索引，以及数据库，将 JSON 文档和对应的 .faiss 向量索引文件配对并加载到内存中

- def retrieve_by_company_name 将查询向量化，在 FAISS 中搜索最近邻。

- def retrieve_all :# 暴力检索，给出所有

### HybridRetriever
- def retrieve_by_company_name ： 先使用向量检索获取较大的候选集 ，然后丢给llm进行相关性评分，然后再得出最后的排名topn

# class LLMReranker:  
它让 LMM 充当裁判，给检索到的文档重新排序打分。

-  def get_rank_for_single_block 这个函数只处理一段文本，询问 LLM 它与 Query 的相关性。

- def get_rank_for_multiple_blocks  这个函数一次性把多个文档（比如4个）发给 LLM，让 LLM 一次性返回4个分数。

- def rerank_documents 主入口，负责分批、并发调用、计算加权分。